# Text Processing in Apache Spark

We can import pyspark or specifically SparkContext to avoid retyping package name

In [ ]:
from pyspark import SparkContext

Do not define spark cotext multiple time

In [ ]:
sc = SparkContext("local[*]", "word_count")

default parallelism equals to:
- num of cores (defined) in local mode
- number of cores defined in oozie job

In [ ]:
sc.defaultParallelism

read file

In [ ]:
!pwd

In [ ]:
!ls data

In [ ]:
lines = sc.textFile("data/shakespeare.txt")
#sc.textFile("hdfs://localhost:8020/input/war-and-peace.txt")

In [ ]:
lines

In [ ]:
type(lines)

RDDs support two types of operations: __transformations__, which create a new dataset from an existing one, and __actions__, which return a value to the driver program after running a computation on the dataset.

All __transformations__ in Spark are __lazy__, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an __action requires a result__ to be returned to the __driver program__. This design enables Spark to run more efficiently. For example, we can realize that a dataset created through map will be used in a reduce and return only the result of the reduce to the driver, rather than the larger mapped dataset.

![](imgs/spark_architecture.png)
check the file type:

In [ ]:
lines.take(10)

In [ ]:
words = lines.flatMap(lambda line: line.split())

In [ ]:
words.take(5)

![](imgs/flatMap1)

In [ ]:
words.count()

In [ ]:
wordCounts = words.countByValue()

In [ ]:
for word, count in wordCounts.items():
    print(word, count)

### Exercises

1. count only words that begin with uppercase
2. Print 5 lines starting with "Romeo" or "Juliet" removing whitespace if necessary.
3. Print 20 most popular words written only with UPPERCASE LETTERS

### (Python) hints

In [ ]:
"  some string with whitespaces \t  ".strip()

In [ ]:
"Jake likes his dog.".startswith("Anne")

In [ ]:
"Jake likes his dog.".startswith("Jake")

In [ ]:
"Anne" or "Jake"  # Don't do: string.startswith(a or b)

In [ ]:
"abc,-".replace(",", "")

In [ ]:
"abc,-".replace(",", "").replace("-", "")

In [ ]:
# Regular expressions
import re
re.findall("[\w]+", "Titus Andronicus Roman-legion")

## Heavy computations and lazy evaluation

In [ ]:
# a very heavy computation
def transform_word(word):
    count = 0
    for i in range(1000):
        count += i
    if len(word)>3:
        return word.lower().replace(' ', 'a').replace('Romeo', 'Julia').replace('b', 'c')
    else:
        return "veryLongWord"

In [ ]:
words

In [ ]:
words_transformed = words.map(lambda x: transform_word(x))

In [ ]:
words_uppercase = words_transformed.map(lambda x: x.upper())

In [ ]:
words_uppercase.getNumPartitions()

In [ ]:
words_repartitioned = words_uppercase.repartition(8)

In [ ]:
words_repartitioned.getNumPartitions()

In [ ]:
word_dict = words_repartitioned.countByValue()

In [ ]:
word_dict

In [ ]:
sorted(word_dict.items(),key=lambda i: i[1],reverse=True)

In [ ]:
words_repartitioned.take(10)

![](imgs/RDD_Operations.png)